In [10]:
from string_grouper import match_strings, match_most_similar, \
    group_similar_strings, compute_pairwise_similarities, \
     StringGrouper

In [14]:
q = """with tw1 as (
SELECT (REPLACE(voyMasterLastName, '.', '') || REPLACE(voyMasterFirstName, '.', '')) name_rec , dv.voyDepartureEDTF, dv.voyArrivalDateEDTF 
FROM das_master dm, das_voyage dv 
WHERE dv.voyMasterID = dm.voyMasterID
ORDER BY dv.voyDepartureEDTF, dv.voyArrivalDateEDTF )
SELECT count(*) as eff, name_rec, (group_concat(SUBSTRING(voyDepartureEDTF, 1,4)  )) liste_dates
FROM tw1
WHERE name_rec IS NOT NULL
group by name_rec
order by eff DESC;
"""
c = cnn.cursor()
c.execute(q)
rc = c.fetchall()

In [15]:
lst = [a[1] for a in rc]
lst[:3]

['VisserPieter', 'VriesJakob', 'PietersenKornelis']

In [16]:
lignes_table = pd.read_sql_query(q, cnn)
lignes_table.head()

,eff,name_rec,liste_dates
0,17,VisserPieter,"1719,1721,1724,1725,1733,1734,1736,1736,1738,1..."
1,16,VriesJakob,"1715,1717,1725,1725,1727,1728,1757,1759,1762,1..."
2,16,PietersenKornelis,"1762,1763,1763,1764,1765,1765,1766,1766,1767,1..."
3,14,WiebeJakob,"1746,1748,1749,1749,1750,1752,1754,1755,1759,1..."
4,13,WitJan,"1614,1690,1693,1695,1696,1699,1708,1709,1740,1..."


In [17]:
# Group customers with similar names:
lignes_table[["group-id", "name_deduped"]]  = \
    group_similar_strings(lignes_table["name_rec"])
# Display the mapping table:
lignes_table.head(8)

,eff,name_rec,liste_dates,group-id,name_deduped
0,17,VisserPieter,"1719,1721,1724,1725,1733,1734,1736,1736,1738,1...",0,VisserPieter
1,16,VriesJakob,"1715,1717,1725,1725,1727,1728,1757,1759,1762,1...",1,VriesJakob
2,16,PietersenKornelis,"1762,1763,1763,1764,1765,1765,1766,1766,1767,1...",2,PietersenKornelis
3,14,WiebeJakob,"1746,1748,1749,1749,1750,1752,1754,1755,1759,1...",3,WiebeJakob
4,13,WitJan,"1614,1690,1693,1695,1696,1699,1708,1709,1740,1...",4,WitJan
5,13,OnkruidJakob,"1699,1700,1708,1710,1715,1716,1718,1718,1719,1...",5,OnkruidJakob
6,12,VetDirk,"1724,1725,1726,1727,1729,1730,1732,1733,1734,1...",6,VetDirk
7,12,HarmeyerGerrit,"1763,1764,1765,1766,1767,1768,1770,1771,1776,1...",7,HarmeyerGerrit


In [18]:
df2 = lignes_table.groupby(['group-id'])['group-id'].count()

In [19]:
df2.sort_values(ascending=False)

group-id
1274    4
1144    3
1421    3
1367    3
1371    3
       ..
987     1
988     1
989     1
990     1
2971    1
Name: group-id, Length: 2831, dtype: int64

In [22]:
lignes_table[["name_rec", "liste_dates", "group-id","name_deduped"]][lignes_table['group-id'] == 1274]

,name_rec,liste_dates,group-id,name_deduped
777,KorneliszKornelis,"1614,1614,1676",1274,JanszKornelis
1274,JanszKornelis,"1629,1695",1274,JanszKornelis
1344,FranszKornelis,"1628,1629",1274,JanszKornelis
2509,HanszKornelis,1613,1274,JanszKornelis
